# Other energy demand

## Import modules

In [1]:
import pandas as pd
from pathlib import Path
from IPython.display import Image, display
import importlib


import src.load_data_manager
import src.transform
import src.yaml_calculator
import src.helper
from src.load_data_manager import LoadDataManager
import config.yaml_to_diagram
from config.yaml_to_diagram import create_dependency_diagram

# Reload the modules to ensure latest version is used
importlib.reload(src.transform);
importlib.reload(src.yaml_calculator);
importlib.reload(src.helper);
importlib.reload(src.load_data_manager);
importlib.reload(config.yaml_to_diagram);

## General

Here we configure the notebook so that all contextual information is loaded and present for the other sections.

TODO: define list with ETLocal keys for which to run this notebook

Give this list as optional argument to helper functions so that if False for a key, then it is not updated in the template.

## Parameters

We first specify general parameters to be used throughout this notebook.

In [2]:
# Select the parent data set. Make sure to use the geo ID. It should be existing in the Dataset Manager.
parent = "nl"

# Specify the year and the reference year for the ETM
year = 2023
year_etm = 2019

# Specify the CSV-separator (presumably either "," or ";")
sep=","

# Specify where the dependency graphs are stored
dep_graph_folder = Path("config","etlocal_keys_final_demand_dependency_outputs", "output")

# Specify where the yaml files with calculation rules are stored
yaml_folder = Path("config","yaml_files_for_etlocal_key_calculation")

## Extract and Transform

Here we define helper functions to be called for multiple cells throughout this specific notebook.

## Extract

In this section:
- We *Load* the relevant data from the data/intermediate folder
- We *Combine* the relevant data by merging municipal and national data

This is handled by the **ETLocalDataModule** which provides a unified interface for all data operations.

The module loads the following data sources:
- The ETLocal template (ETLocal_template_empty.csv)
- Klimaatmonitor data - both municipal and national levels with metadata:
  - km_source_data_converted.csv (municipal data)
  - km_meta_data_converted.csv (municipal metadata)
  - km_national_source_data_converted.csv (national data)
  - km_national_meta_data_converted.csv (national metadata)
- Transport research data (transport_research_cleaned.csv)
- Miscellaneous data (miscellaneous_data_analysis.csv)
- ETM query results (etm_query_combined.csv)

The module:
- Combines municipal-level data from different sources
- Combines national-level data from different sources
- Merges the municipal and national datasets into a single DataFrame
- Caches loaded data for improved performance

In [3]:
data_manager = LoadDataManager()
combined_data = data_manager.combine_all_data(
    include_miscellaneous=True,
    include_etm_queries=True,
)
combined_data

#If you want to view etm_queries, uncomment:
# etm_queries = data_manager.load_etm_queries_old()

,Gemeentenaam,ProvinciecodePV,Provincienaam,no_inhabitants,no_residences,no_cars,total_energy_tj,tp_total_energy_tj,total_electricity_tj,total_gas_heat_tj_temp_corrected,...,mq_buildings_final_demand_for_lighting_electricity_buildings_lighting_led_electricity_parent_share,mq_buildings_final_demand_for_cooling_electricity_buildings_cooling_airconditioning_electricity_parent_share,mq_buildings_final_demand_for_cooling_electricity_buildings_cooling_heatpump_air_water_electricity_parent_share,mq_buildings_final_demand_for_cooling_electricity_buildings_cooling_collective_heatpump_water_water_ts_electricity_parent_share,mq_input_percentage_of_lt_steam_hot_water_buildings_final_demand_steam_hot_water,mq_input_percentage_of_mt_steam_hot_water_buildings_final_demand_steam_hot_water,mq_input_percentage_of_ht_steam_hot_water_buildings_final_demand_steam_hot_water,mq_input_percentage_of_lt_steam_hot_water_households_final_demand_steam_hot_water,mq_input_percentage_of_mt_steam_hot_water_households_final_demand_steam_hot_water,mq_input_percentage_of_ht_steam_hot_water_households_final_demand_steam_hot_water
GM0014,Groningen,PV20,Groningen,238147.0,122715.0,NaN,15294.0,4045.0,3218.0,8031.0,...,0.460186,0.918529,0.071604,0.009867,0.0,0.0,1.0,0.0,0.0,1.0
GM0034,Almere,PV24,Flevoland,222825.0,90690.0,NaN,10680.0,3869.0,2654.0,4289.0,...,0.460186,0.918529,0.071604,0.009867,0.0,0.0,1.0,0.0,0.0,1.0
GM0037,Stadskanaal,PV20,Groningen,32135.0,15177.0,NaN,1759.0,534.0,409.0,815.0,...,0.460186,0.918529,0.071604,0.009867,0.0,0.0,1.0,0.0,0.0,1.0
GM0047,Veendam,PV20,Groningen,27616.0,13013.0,NaN,0.0,580.0,786.0,0.0,...,0.460186,0.918529,0.071604,0.009867,0.0,0.0,1.0,0.0,0.0,1.0
GM0050,Zeewolde,PV24,Flevoland,23692.0,9354.0,NaN,3139.0,1581.0,765.0,794.0,...,0.460186,0.918529,0.071604,0.009867,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GM1979,Eemsdelta,PV20,Groningen,45394.0,23587.0,NaN,0.0,1049.0,2026.0,0.0,...,0.460186,0.918529,0.071604,0.009867,0.0,0.0,1.0,0.0,0.0,1.0
GM1980,Dijk en Waard,PV27,Noord-Holland,88985.0,37459.0,NaN,5168.0,1350.0,1110.0,2708.0,...,0.460186,0.918529,0.071604,0.009867,0.0,0.0,1.0,0.0,0.0,1.0
GM1982,Land van Cuijk,PV30,Noord-Brabant,91423.0,39868.0,NaN,9022.0,3179.0,2073.0,3769.0,...,0.460186,0.918529,0.071604,0.009867,0.0,0.0,1.0,0.0,0.0,1.0
GM1991,Maashorst,PV30,Noord-Brabant,58934.0,26012.0,NaN,4607.0,1761.0,1191.0,1655.0,...,0.460186,0.918529,0.071604,0.009867,0.0,0.0,1.0,0.0,0.0,1.0


In [4]:
# Load the template
df_template_local = data_manager.load_template(group = 'other')
df_template_local

unit  \
geo_id group subgroup            key                                                      
GM1680 other other_energy_demand other_final_demand_electricity_demand               TJ   
                                 other_final_demand_network_gas_demand               TJ   
                                 input_other_final_demand_steam_hot_water_demand     TJ   
                                 other_final_demand_wood_pellets_demand              TJ   
                                 other_final_demand_coal_demand                      TJ   
...                                                                                 ...   
GM0193 other other_energy_demand input_other_final_demand_steam_hot_water_demand     TJ   
                                 other_final_demand_wood_pellets_demand              TJ   
                                 other_final_demand_coal_demand                      TJ   
                                 other_final_demand_crude_oil_demand                 TJ   
                                 other_final_demand_crude_oil_non_energetic_demand   TJ   

                                                                                    value  \
geo_id group subgroup            key                                                        
GM1680 other other_energy_demand other_final_demand_electricity_demand                NaN   
                                 other_final_demand_network_gas_demand                NaN   
                                 input_other_final_demand_steam_hot_water_demand      NaN   
                                 other_final_demand_wood_pellets_demand               NaN   
                                 other_final_demand_coal_demand                       NaN   
...                                                                                   ...   
GM0193 other other_energy_demand input_other_final_demand_steam_hot_water_demand      NaN   
                                 other_final_demand_wood_pellets_demand               NaN   
                                 other_final_demand_coal_demand                       NaN   
                                 other_final_demand_crude_oil_demand                  NaN   
                                 other_final_demand_crude_oil_non_energetic_demand    NaN   

                                                                                   commit  
geo_id group subgroup            key                                                       
GM1680 other other_energy_demand other_final_demand_electricity_demand                nan  
                                 other_final_demand_network_gas_demand                nan  
                                 input_other_final_demand_steam_hot_water_demand      nan  
                                 other_final_demand_wood_pellets_demand               nan  
                                 other_final_demand_coal_demand                       nan  
...                                                                                   ...  
GM0193 other other_energy_demand input_other_final_demand_steam_hot_water_demand      nan  
                                 other_final_demand_wood_pellets_demand               nan  
                                 other_final_demand_coal_demand                       nan  
                                 other_final_demand_crude_oil_demand                  nan  
                                 other_final_demand_crude_oil_non_energetic_demand    nan  

[2394 rows x 3 columns]

In this section we import the relevant dataframes from the data/intermediate folder.

## Run YamlCalculator

### Keys set to zero

In [5]:
input_keys_zero = [
'input_other_final_demand_steam_hot_water_demand',
'other_final_demand_coal_demand',
'other_final_demand_crude_oil_demand',
'other_final_demand_crude_oil_non_energetic_demand',
'other_final_demand_electricity_demand',
'other_final_demand_network_gas_demand',
'other_final_demand_wood_pellets_demand'
]

In [6]:
# Move later to src.helper
def set_key_to_zero(etlocal_key, df_template_local):
    etlocal_key = etlocal_key
    df_template_local = src.helper.fill_template_with_uniform_value_etlocal_data(etlocal_key, 0, df_template_local)
    return df_template_local

# df_template_local = data_manager.load_template()   
for i in range(len(input_keys_zero)):
    df_template_local = set_key_to_zero(input_keys_zero[i], df_template_local)

In [7]:
df_template_local.loc[(slice(None), slice(None), slice(None), 'input_other_final_demand_steam_hot_water_demand')]
# df_template_local

,,,unit,value,commit
geo_id,group,subgroup,,,
GM0014,other,other_energy_demand,TJ,0.0,No data available. Set to 0 for all municipali...
GM0034,other,other_energy_demand,TJ,0.0,No data available. Set to 0 for all municipali...
GM0037,other,other_energy_demand,TJ,0.0,No data available. Set to 0 for all municipali...
GM0047,other,other_energy_demand,TJ,0.0,No data available. Set to 0 for all municipali...
GM0050,other,other_energy_demand,TJ,0.0,No data available. Set to 0 for all municipali...
...,...,...,...,...,...
GM1979,other,other_energy_demand,TJ,0.0,No data available. Set to 0 for all municipali...
GM1980,other,other_energy_demand,TJ,0.0,No data available. Set to 0 for all municipali...
GM1982,other,other_energy_demand,TJ,0.0,No data available. Set to 0 for all municipali...


## Export

In [8]:
# Write the filled template to a CSV file
path_output = Path("data", "processed", f"etlocal_template_other_energy_filled.csv")
df_template_local.to_csv(path_output, sep=sep)
